#Dense Neural Network Experiments

## 1. Initial Setup

1.1. Begin by importing the necessary libraries. Seeds are set for reproducibility.

In [ ]:
import numpy as np
np.random.seed(1234)
from keras.models import Sequential
import tensorflow as tf
tf.random.set_seed(123)
from keras.layers import Dense, Activation
from keras.utils import np_utils
from keras.callbacks import EarlyStopping

1.2. Load and preprocess the MNIST data set.

In [ ]:
from keras.datasets import mnist

# Load and split data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#Transform 3D X data into 2D 
X_train_flatten = X_train.reshape(X_train.shape[0], X_train.shape[1] * X_train.shape[2])
X_test_flatten = X_test.reshape(X_test.shape[0], X_test.shape[1] * X_test.shape[2])

#Normalise X data elements to all be in range [0,1]
X_train_flatten = X_train_flatten.astype('float32')
X_test_flatten = X_test_flatten.astype('float32')
X_train_flatten /= 255
X_test_flatten /= 255

#Adjust y data so that numerical categorical labels become one-hot vectors of size 10
Y_train_class = np_utils.to_categorical(y_train, 10)
Y_test_class = np_utils.to_categorical(y_test, 10)

1.3. Create a function to generate a 3-layer dense neural network, with parameterisable structure, to perform the prediction task.

In [ ]:
INPUT_DIM = 784

def create_network(L1_neurons, L2_neurons, **kwargs):
  L1_activation = 'relu'
  L2_activation = 'relu'
  L3_activation = 'softmax'

  L3_neurons = 10

  if 'L1_activation' in kwargs.keys():
    L1_activation = kwargs['L1_activation']
  if 'L2_activation' in kwargs.keys():
    L2_activation = kwargs['L2_activation']
  if 'L3_activation' in kwargs.keys():
    L3_activation = kwargs['L3_activation']
  if 'L3_neurons' in kwargs.keys():
    L3_neurons = kwargs['L3_neurons']

  model = Sequential()
  model.add(Dense(L1_neurons, activation=L1_activation, input_shape=(INPUT_DIM,), name='L1'))
  model.add(Dense(L2_neurons, activation=L2_activation, name='L2'))
  model.add(Dense(L3_neurons, activation=L3_activation, name='L3'))

  return model

#Test the function
test_model = create_network(64, 32, L3_neurons = 25)
test_model.summary()

1.4. Write a function which will accept the hyperparameters, generate the neural network and evaluate it against the data set.

In [ ]:
def evaluate(L1_neurons, L2_neurons, **kwargs):
  model = create_network(int(L1_neurons), int(L2_neurons), **kwargs)

  loss_function = 'categorical_crossentropy'
  optimizer = 'adam'
  if 'loss_function' in kwargs.keys():
    loss_function = kwargs['loss_function']
  if 'optimizer' in kwargs.keys():
    optimizer = kwargs['optimizer']

  model.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])

  batch_size = 32
  if 'batch_size' in kwargs.keys():
    batch_size = int(kwargs['batch_size'])

  history = None
  if 'epochs' in kwargs.keys():
    epochs = int(kwargs['epochs'])
    history = model.fit(X_train_flatten, Y_train_class, batch_size=batch_size, epochs=epochs, verbose=0)
  else:
    early_stopping = EarlyStopping(monitor='loss', patience=5)
    history = model.fit(X_train_flatten, Y_train_class, epochs=1000, batch_size=batch_size, callbacks=[early_stopping], verbose=0)

  score = model.evaluate(X_test_flatten, Y_test_class, verbose=0)

  return score[0]

#Test the function
evaluate(64, 32, epochs=1)

## 2. Hyperparameter Optimisation Experiments

### 2.A. Setup

#### 2.A.1. Import GitHub Repo

In [ ]:
!git clone https://github.com/aamanrebello/HTVTC-Testing-Framework.git

#Enable importing code from parent directory
import os, sys
final_HTVTC = os.path.abspath('./HTVTC-Testing-Framework/final-HTVTC')
sys.path.insert(1, final_HTVTC)
traditional_methods = os.path.abspath('./HTVTC-Testing-Framework/traditional-methods')
sys.path.insert(1, traditional_methods)
root = os.path.abspath('./HTVTC-Testing-Framework')
sys.path.insert(1, root)

#### 2.A.2. Setup for HTVTC

In [ ]:
!pip install tensorly

#### 2.A.3. Setup for Random Search, BO-TPE, CMA-ES

In [ ]:
!pip install optuna
import optuna

#### 2.A.4. Setup for Hyperband

In [ ]:
!pip install optuna
import optuna

def evaluate_with_budget(L1_neurons, L2_neurons, budget_fraction=1.0, **kwargs):
  model = create_network(int(L1_neurons), int(L2_neurons), **kwargs)

  loss_function = 'categorical_crossentropy'
  optimizer = 'adam'
  if 'loss_function' in kwargs.keys():
    loss_function = kwargs['loss_function']
  if 'optimizer' in kwargs.keys():
    optimizer = kwargs['optimizer']

  model.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])

  batch_size = 32
  if 'batch_size' in kwargs.keys():
    batch_size = int(kwargs['batch_size'])

  history = None
  training_size = int(budget_fraction*len(X_train_flatten))
  X_train_trunc = X_train_flatten[:training_size]
  Y_train_trunc = Y_train_class[:training_size]

  if 'epochs' in kwargs.keys():
    epochs = int(kwargs['epochs'])
    history = model.fit(X_train_trunc, Y_train_trunc, batch_size=batch_size, epochs=epochs, verbose=0)
  else:
    early_stopping = EarlyStopping(monitor='loss', patience=5)
    history = model.fit(X_train_trunc, Y_train_trunc, epochs=1000, batch_size=batch_size, callbacks=[early_stopping], verbose=0)

  score = model.evaluate(X_test_flatten, Y_test_class, verbose=0)

  return score[0]

#### 2.A.5. Setup for BOHB

In [ ]:
!pip install hpbandster

def evaluate_with_budget(L1_neurons, L2_neurons, budget_fraction=1.0, **kwargs):
  model = create_network(int(L1_neurons), int(L2_neurons), **kwargs)

  loss_function = 'categorical_crossentropy'
  optimizer = 'adam'
  if 'loss_function' in kwargs.keys():
    loss_function = kwargs['loss_function']
  if 'optimizer' in kwargs.keys():
    optimizer = kwargs['optimizer']

  model.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])

  batch_size = 32
  if 'batch_size' in kwargs.keys():
    batch_size = int(kwargs['batch_size'])

  history = None
  training_size = int(budget_fraction*len(X_train_flatten))
  X_train_trunc = X_train_flatten[:training_size]
  Y_train_trunc = Y_train_class[:training_size]

  if 'epochs' in kwargs.keys():
    epochs = int(kwargs['epochs'])
    history = model.fit(X_train_trunc, Y_train_trunc, batch_size=batch_size, epochs=epochs, verbose=0)
  else:
    early_stopping = EarlyStopping(monitor='loss', patience=5)
    history = model.fit(X_train_trunc, Y_train_trunc, epochs=1000, batch_size=batch_size, callbacks=[early_stopping], verbose=0)

  score = model.evaluate(X_test_flatten, Y_test_class, verbose=0)

  return score[0]

#### 2.A.6. Setup for BO-GP

In [ ]:
!pip install git+https://github.com/fmfn/BayesianOptimization

### 2.B. Hyperparameter Optimisation Experiments

#### 2.B.1. HTVTC

In [ ]:
from finalAlgoImplementation import final_HTVTC
import regressionmetrics
import classificationmetrics

metric = classificationmetrics.indicatorFunction

quantity = 'EXEC-TIME'

func = evaluate

#Start timer/memory profiler/CPU timer
a = None
start_time = None
if quantity == 'EXEC-TIME':
    import time
    start_time = time.perf_counter_ns()
elif quantity == 'CPU-TIME':
    import time
    start_time = time.process_time_ns()
elif quantity == 'MAX-MEMORY':
    import tracemalloc
    tracemalloc.start()

ranges_dict = {
    'L1_neurons': {
        'type': 'INTEGER',
        'start': 20.0,
        'end': 101.0,
        'interval': 20.0
        },
    'L2_neurons': {
        'type': 'INTEGER',
        'start': 10.0,
        'end': 51.00,
        'interval': 20.0
        },
    'L1_activation': {
        'type': 'CATEGORICAL',
        'values': ['relu', 'tanh', 'sigmoid']
        },
    'L2_activation': {
        'type': 'CATEGORICAL',
        'values': ['relu', 'tanh', 'sigmoid']
        },
    'epochs': {
        'type': 'CATEGORICAL',
        'values': [1]
    }
  }

recommended_combination, history = final_HTVTC(eval_func=func, ranges_dict=ranges_dict, metric=metric, max_completion_cycles=6, max_size_gridsearch=50)

#End timer/memory profiler/CPU timer
result = None
if quantity == 'EXEC-TIME':
    end_time = time.perf_counter_ns()
    result = end_time - start_time
elif quantity == 'CPU-TIME':
    end_time = time.process_time_ns()
    result = end_time - start_time
elif quantity == 'MAX-MEMORY':
    _, result = tracemalloc.get_traced_memory()
    tracemalloc.stop()

#Find the true loss for the selcted combination
true_value = func(metric=metric, **recommended_combination)

print(f'hyperparameters: {recommended_combination}')
print(f'history: {history}')
print(f'True value: {true_value}')
print(f'{quantity}: {result}')

#### 2.B.2. Random Search

In [ ]:
from optuna.samplers import RandomSampler

quantity = 'EXEC-TIME'
func = evaluate

def objective(trial):
    L1_neurons = trial.suggest_int("L1_neurons", 20, 101, step=1)
    L2_neurons = trial.suggest_int("L2_neurons", 10, 51, step=1)
    L1_activation = trial.suggest_categorical("L1_activation", ['relu', 'tanh', 'sigmoid'])
    L2_activation = trial.suggest_categorical("L2_activation", ['relu', 'tanh', 'sigmoid'])
    
    return func(L1_neurons=L1_neurons, L2_neurons=L2_neurons, L1_activation=L1_activation, L2_activation=L2_activation, epochs=1)

#Start timer/memory profiler/CPU timer
start_time = None
if quantity == 'EXEC-TIME':
    import time
    start_time = time.perf_counter_ns()
elif quantity == 'CPU-TIME':
    import time
    start_time = time.process_time_ns()
elif quantity == 'MAX-MEMORY':
    import tracemalloc
    tracemalloc.start()

optuna.logging.set_verbosity(optuna.logging.FATAL)
study = optuna.create_study(sampler=RandomSampler())
study.optimize(objective, n_trials=115)

result = None
if quantity == 'EXEC-TIME':
    end_time = time.perf_counter_ns()
    result = end_time - start_time
elif quantity == 'CPU-TIME':
    end_time = time.process_time_ns()
    result = end_time - start_time
elif quantity == 'MAX-MEMORY':
    _, result = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
print('\n\n\n')
print(f'Number of trials: {len(study.trials)}')
print(f'Best trial: {study.best_trial}')
print(f'{quantity}: {result}')

#### 2.B.3. BO-TPE

In [ ]:
from optuna.samplers import TPESampler

quantity = 'EXEC-TIME'
func = evaluate


def objective(trial):
    L1_neurons = trial.suggest_int("L1_neurons", 20, 101, step=1)
    L2_neurons = trial.suggest_int("L2_neurons", 10, 51, step=1)
    L1_activation = trial.suggest_categorical("L1_activation", ['relu', 'tanh', 'sigmoid'])
    L2_activation = trial.suggest_categorical("L2_activation", ['relu', 'tanh', 'sigmoid'])
    
    return func(L1_neurons=L1_neurons, L2_neurons=L2_neurons, L1_activation=L1_activation, L2_activation=L2_activation, epochs=1)

#Start timer/memory profiler/CPU timer
start_time = None
if quantity == 'EXEC-TIME':
    import time
    start_time = time.perf_counter_ns()
elif quantity == 'CPU-TIME':
    import time
    start_time = time.process_time_ns()
elif quantity == 'MAX-MEMORY':
    import tracemalloc
    tracemalloc.start()

optuna.logging.set_verbosity(optuna.logging.FATAL)
study = optuna.create_study(sampler=TPESampler())
study.optimize(objective, n_trials=115)

#End timer/memory profiler/CPU timer
result = None
if quantity == 'EXEC-TIME':
    end_time = time.perf_counter_ns()
    result = end_time - start_time
elif quantity == 'CPU-TIME':
    end_time = time.process_time_ns()
    result = end_time - start_time
elif quantity == 'MAX-MEMORY':
    _, result = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
print('\n\n\n')
print(f'Number of trials: {len(study.trials)}')
print(f'Best trial: {study.best_trial}')
print(f'{quantity}: {result}')

#### 2.B.4. CMA-ES

In [ ]:
quantity = 'EXEC-TIME'
func = evaluate

def objective(trial):
    L1_neurons = trial.suggest_int("L1_neurons", 20, 101, step=1)
    L2_neurons = trial.suggest_int("L2_neurons", 10, 51, step=1)
    L1_activation = trial.suggest_categorical("L1_activation", ['relu', 'tanh', 'sigmoid'])
    L2_activation = trial.suggest_categorical("L2_activation", ['relu', 'tanh', 'sigmoid'])
    
    return func(L1_neurons=L1_neurons, L2_neurons=L2_neurons, L1_activation=L1_activation, L2_activation=L2_activation, epochs=1)

#Start timer/memory profiler/CPU timer
start_time = None
if quantity == 'EXEC-TIME':
    import time
    start_time = time.perf_counter_ns()
elif quantity == 'CPU-TIME':
    import time
    start_time = time.process_time_ns()
elif quantity == 'MAX-MEMORY':
    import tracemalloc
    tracemalloc.start()

optuna.logging.set_verbosity(optuna.logging.FATAL)
sampler = optuna.samplers.CmaEsSampler()
study = optuna.create_study(sampler=sampler)
study.optimize(objective, n_trials=115)
#resource_usage = getrusage(RUSAGE_SELF)

#End timer/memory profiler/CPU timer
result = None
if quantity == 'EXEC-TIME':
    end_time = time.perf_counter_ns()
    result = end_time - start_time
elif quantity == 'CPU-TIME':
    end_time = time.process_time_ns()
    result = end_time - start_time
elif quantity == 'MAX-MEMORY':
    _, result = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
print('\n\n\n')
print(f'Number of trials: {len(study.trials)}')
print(f'Best trial: {study.best_trial}')
print(f'{quantity}: {result}')

#### 2.B.5. Hyperband

In [ ]:
from commonfunctions import generate_range

quantity = 'EXEC-TIME'
resolution = 0.2

func = evaluate_with_budget

def obtain_hyperparameters(trial):
    L1_neurons = trial.suggest_int("L1_neurons", 20, 101, step=1)
    L2_neurons = trial.suggest_int("L2_neurons", 10, 51, step=1)
    L1_activation = trial.suggest_categorical("L1_activation", ['relu', 'tanh', 'sigmoid'])
    L2_activation = trial.suggest_categorical("L2_activation", ['relu', 'tanh', 'sigmoid'])

    return L1_neurons, L2_neurons, L1_activation, L2_activation


def objective(trial):
    L1_neurons, L2_neurons, L1_activation, L2_activation = obtain_hyperparameters(trial)
    metric_value = None

    for fraction in generate_range(resolution,1,resolution):
        metric_value = func(L1_neurons=L1_neurons, L2_neurons=L2_neurons, budget_fraction=fraction, L1_activation=L1_activation, L2_activation=L2_activation, epochs=1)
        #Check for pruning
        trial.report(metric_value, fraction)
        if trial.should_prune():
            print('=======================================================================================================')
            raise optuna.TrialPruned()

    #Would return the metric for fully trained model (on full dataset)
    return metric_value
    

#Start timer/memory profiler/CPU timer
start_time = None
if quantity == 'EXEC-TIME':
    import time
    start_time = time.perf_counter_ns()
elif quantity == 'CPU-TIME':
    import time
    start_time = time.process_time_ns()
elif quantity == 'MAX-MEMORY':
    import tracemalloc
    tracemalloc.start()

optuna.logging.set_verbosity(optuna.logging.FATAL)
study = optuna.create_study(
    direction="minimize",
    pruner=optuna.pruners.HyperbandPruner(
        min_resource=resolution, max_resource=1, reduction_factor=2
    ),
)
study.optimize(objective, n_trials=30)

result = None
if quantity == 'EXEC-TIME':
    end_time = time.perf_counter_ns()
    result = end_time - start_time
elif quantity == 'CPU-TIME':
    end_time = time.process_time_ns()
    result = end_time - start_time
elif quantity == 'MAX-MEMORY':
    _, result = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
print('\n\n\n')
print(f'Number of trials: {len(study.trials)}')
print(f'Best trial: {study.best_trial}')
print(f'{quantity}: {result}')

#### 2.B.6. BOHB

In [ ]:
import ConfigSpace as CS
import ConfigSpace.hyperparameters as CSH
import hpbandster.core.nameserver as hpns
import hpbandster.core.result as hpres
from hpbandster.core.worker import Worker
from hpbandster.examples.commons import MyWorker
from hpbandster.optimizers import BOHB as BOHB

#To hide logs
import logging
logObj = logging.getLogger('noOutput')
logObj.setLevel(100)

#To hide warnings
import warnings
warnings.filterwarnings("ignore")

func = evaluate_with_budget

#Define the worker
class MyWorker(Worker):

    def __init__(self, *args, sleep_interval=0, **kwargs):
        super().__init__(*args, **kwargs)

        self.sleep_interval = sleep_interval

    def compute(self, config, budget, **kwargs):
        res = func(**config, budget_fraction=budget, epochs=1)
        
        return({
                    'loss': res,
                    'info': res
                })
    
    @staticmethod
    def get_configspace():
        cs = CS.ConfigurationSpace()
        L1_activation = CSH.CategoricalHyperparameter("L1_activation", ['relu', 'tanh', 'sigmoid'])
        L2_activation = CSH.CategoricalHyperparameter("L2_activation", ['relu', 'tanh', 'sigmoid'])
        cs.add_hyperparameters([L1_activation, L2_activation])

        L1_neurons = CSH.UniformIntegerHyperparameter('L1_neurons', lower=20, upper=101)
        L2_neurons = CSH.UniformIntegerHyperparameter('L2_neurons', lower=10, upper=51)
        cs.add_hyperparameters([L1_neurons, L2_neurons])

        return cs

#Setup nameserver
NS = hpns.NameServer(run_id='dnn', host='127.0.0.1', port=None)
NS.start()

#Start a worker
w = MyWorker(sleep_interval = 0, nameserver='127.0.0.1',run_id='dnn', logger=logObj)
w.run(background=True)

quantity = 'EXEC-TIME'

#Start timer/memory profiler/CPU timer
start_time = None
if quantity == 'EXEC-TIME':
    import time
    start_time = time.perf_counter_ns()
elif quantity == 'CPU-TIME':
    import time
    start_time = time.process_time_ns()
elif quantity == 'MAX-MEMORY':
    import tracemalloc
    tracemalloc.start()

#Run the optimiser
MAX_BUDGET = 1.0
MIN_BUDGET = 0.2
bohb = BOHB(  configspace = w.get_configspace(),
              run_id = 'dnn', nameserver='127.0.0.1',
              min_budget=MIN_BUDGET, max_budget=MAX_BUDGET,
              logger=logObj
           )
res = bohb.run(n_iterations=50)

#End timer/memory profiler/CPU timer
quantity_result = None
if quantity == 'EXEC-TIME':
    end_time = time.perf_counter_ns()
    quantity_result = end_time - start_time
elif quantity == 'CPU-TIME':
    end_time = time.process_time_ns()
    quantity_result = end_time - start_time
elif quantity == 'MAX-MEMORY':
    _, quantity_result = tracemalloc.get_traced_memory()
    tracemalloc.stop()

#Shutdown
bohb.shutdown(shutdown_workers=True)
NS.shutdown()

id2config = res.get_id2config_mapping()
inc_id = res.get_incumbent_id()
inc_runs = res.get_runs_by_id(inc_id)
inc_run = inc_runs[-1]

print('Best found configuration:', id2config[inc_id]['config'])
print(f'Validation loss: {inc_run.loss}')
print('A total of %i unique configurations were sampled.' % len(id2config.keys()))
print('A total of %i runs were executed.' % len(res.get_all_runs()))
print('Total budget corresponds to %.1f full function evaluations.'%(sum([r.budget for r in res.get_all_runs()])/MAX_BUDGET))
print(f'{quantity}: {quantity_result}')

#### 2.B.7. BO-GP

In [ ]:
from bayes_opt import BayesianOptimization

quantity = 'EXEC-TIME'
trials = 95
pval = 1

func = evaluate

def classify_activation(value):
   if value < 5:
      return 'relu'
   elif value < 10:
      return 'tanh'
   else:
      return 'sigmoid'

def objective(L1_neurons, L2_neurons, L1_activation, L2_activation):
    L1_neu_int = int(L1_neurons)
    L2_neu_int = int(L2_neurons)

    L1_act_str = classify_activation(L1_activation)
    L2_act_str = classify_activation(L2_activation)

    #subtract from 1 because the library only supports maximise
    return pval - func(L1_neurons=L1_neu_int, L2_neurons=L2_neu_int, L1_activation=L1_act_str, L2_activation=L2_act_str, epochs=1)

#Start timer/memory profiler/CPU timer
start_time = None
if quantity == 'EXEC-TIME':
    import time
    start_time = time.perf_counter_ns()
elif quantity == 'CPU-TIME':
    import time
    start_time = time.process_time_ns()
elif quantity == 'MAX-MEMORY':
    import tracemalloc
    tracemalloc.start()

#Begin optimisation
pbounds = {'L1_neurons': (20, 101), 'L2_neurons': (10, 51), 'L1_activation': (0, 15), 'L2_activation': (0, 15)}

optimizer = BayesianOptimization(
    f=objective,
    pbounds=pbounds,
    random_state=1,
    verbose = 0
)

optimizer.maximize(
    init_points=10,
    n_iter=trials,
)

result = None
if quantity == 'EXEC-TIME':
    end_time = time.perf_counter_ns()
    result = end_time - start_time
elif quantity == 'CPU-TIME':
    end_time = time.process_time_ns()
    result = end_time - start_time
elif quantity == 'MAX-MEMORY':
    _, result = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
print('\n\n\n')
best = optimizer.max
best_params = best['params']
best_score = pval - best['target']
print(f'Number of trials: {trials}')
print(f'Best params: {best_params}')
print(f'Best score: {best_score}')
print(f'{quantity}: {result}')

### 2.C. Re-evaluate Specific Hyperparameter Combinations

In [ ]:
#Initialise the desired combination here
combination_to_test = {}
combination_to_test['epochs'] = 1

#Evaluate 5 times
true_value1 = evaluate(**combination_to_test)
true_value2 = evaluate(**combination_to_test)
true_value3 = evaluate(**combination_to_test)
true_value4 = evaluate(**combination_to_test)
true_value5 = evaluate(**combination_to_test)

average = sum([true_value1, true_value2, true_value3, true_value4, true_value5])/5

print(f'hyperparameters: {combination_to_test}')
print(f'True values: {true_value1}, {true_value2}, {true_value3}, {true_value4}, {true_value5}')
print(f'mean: {average}')

## 3. Display Backend Specifications

In [ ]:
!lscpu

In [ ]:
!nvidia-smi -L